In this tutorial, we will fold a protein structure using a very simple algorithm in PyRosetta, and compare the folded structure with the solved crystal structure of the protein. 


Modified from: MIT How to Grow Almost anything MAS.S621


Files:

https://drive.google.com/drive/folders/1uugeX7yp1BAg6Ftcm3jbV-ry2SRNc-qC?usp=sharing

# Installing python libraries

In [ ]:
!pip install pyrosettacolabsetup
!pip install py3Dmol
!pip install nglview

import sys
sys.path.append("/usr/local/lib/python3.9/site-packages")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.7 MB 8.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.6 MB 31.2 MB/s 
  Created wheel for nglview: filename=nglview-3.0.3-py3-none-any.whl size=8057551 sha256=af49f2ee5ab14dfe473d02b00dc8f301a3717ba695a276f597081d7479540bca
  Stored in directory: /root/.cache/pip/wheels/01/0c/49/c6f79d8edba8fe89752bf20de2d99040bfa57db0548975c5d5
Successfully built nglview


# Import necessary python libraries

In [ ]:
import glob
import pandas as pd
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()
from pyrosetta import *
init()

Mounted at /content/google_drive
Looking for compatible PyRosetta wheel file at google-drive/PyRosetta/colab.bin/wheels...
Found compatible wheel: /content/google_drive/MyDrive/PyRosetta/colab.bin/wheels//content/google_drive/MyDrive/PyRosetta/colab.bin/wheels/pyrosetta-2022.34+release.9ec33c9fe00-cp37-cp37m-linux_x86_64.whl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


PyRosetta-4 2022 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2022.34+release.9ec33c9fe00427e5b1b0393e84cd8062c07f104c 2022-08-24T09:49:27] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python37.ubuntu r328 2022.34+release.9ec33c9fe00 9ec33c9fe00427e5b1b0393e84cd8062c07f104c http://www.pyrosetta.org 2022-08-24T09:49:27
core.init: command: PyRosetta -ex1 -

### Setting up the score function

In [ ]:
centroid_sfxn = pyrosetta.create_score_function("score3")
full_atom_sfxn = get_score_function()

basic.io.database: Database file opened: scoring/score_functions/EnvPairPotential/env_log.txt
basic.io.database: Database file opened: scoring/score_functions/EnvPairPotential/cbeta_den.txt
basic.io.database: Database file opened: scoring/score_functions/EnvPairPotential/pair_log.txt
basic.io.database: Database file opened: scoring/score_functions/EnvPairPotential/cenpack_log.txt
basic.io.database: Database file opened: scoring/score_functions/SecondaryStructurePotential/phi.theta.36.HS.resmooth
basic.io.database: Database file opened: scoring/score_functions/SecondaryStructurePotential/phi.theta.36.SS.resmooth
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: F

### Loading the native (solved crystal) structure

In [ ]:
native_pose = pose_from_pdb('/content/AMHR2.clean.pdb')

core.import_pose.import_pose: File '/content/AMHR2.clean.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue PRO:CtermProteinFull 109
core.conformation.Conformation: Found disulfide between residues 9 46
core.conformation.Conformation: current variant for 9 CYS
core.conformation.Conformation: current variant for 46 CYS
core.conformation.Conformation: current variant for 9 CYD
core.conformation.Conformation: current variant for 46 CYD
core.conformation.Conformation: Found disulfide between residues 40 64
core.conformation.Conformation: current variant for 40 CYS
core.conformation.Conformation: current variant for 64 CYS
core.conformation.Conformation: current variant for 40 CYD
core.conformation.Conformation: current variant for 64 CYD
core.conformation.Conformation: Found disulfide between residues 45 72
core.conformation.Conformation: current variant for 45 CYS
core.conformation.Conformation: current variant fo

In [ ]:
native_pose.sequence()

'HMPPNRRTCVFFEAPGVRGSTKTLGELLDTGTELPRAIRCLYSRCCFGIWNLTQDRAQVEMQGCRDSDEPGCESLHCDPSPRAHPSPGSTLFTCSCGTDFCNANYSHLP'

# Make a new pose object using the sequence information

In [ ]:
pose = pose_from_sequence(native_pose.sequence())
sequence_pose = Pose()
sequence_pose.assign(pose)

## Defining movers to switch from full-atom to centroid representation

In [ ]:
to_centroid = SwitchResidueTypeSetMover('centroid')
to_full_atom = SwitchResidueTypeSetMover('fa_standard')

In [ ]:
to_full_atom.apply(sequence_pose)
print('Full Atom Score:', full_atom_sfxn(sequence_pose))
to_centroid.apply(sequence_pose)
print('Centroid Score:', centroid_sfxn(sequence_pose))

core.util.switchresiduetypeset: [ WARNING ] When switching to a fa_standard ResidueTypeSet:  Pose already contains fa_standard ResidueTypes.
basic.io.database: Database file opened: scoring/score_functions/elec_cp_reps.dat
core.scoring.elec.util: Read 40 countpair representative atoms
core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /usr/local/lib/python3.7/dist-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: Using Dunbrack library binary file '/usr/local/lib/python3.7/dist-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin'.
core.pack.dunbrack.RotamerLibrary: Dunbrack 2010 library took 0.356997 seconds to load from binary
Full Atom Score: 29842.709056933694
core.chemical.GlobalResid

Setting up the fragment libraries

URL for fragment libraries - http://old.robetta.org/fragmentsubmit.jsp **bold text**

In [ ]:
# Loading the files with the pre-computed fragmets
long_frag_filename = 'AMHR2_9_fragments.txt'
long_frag_length = 9
short_frag_filename = 'AMHR2_3_fragments.txt'
short_frag_length = 3

# Defining parameters of the folding algorithm
long_inserts=5  # How many 9-fragment pieces to insest during the search
short_inserts=10 # How many 3-fragment pieces to insest during the search

kT = 3.0 # Simulated Annealing temperature
cycles = 50 # How many cycles of Monte Carlo search to run
jobs = 5 # How many trajectories in parallel to compute.
job_output = 'outputs/AMHR2/decoy' # The prefix of the filenames to store the results

In [ ]:
movemap = MoveMap()
movemap.set_bb(True)

#define the constant length fragset class - we will do this for 3 mer and 9 mer
fragset_long = rosetta.core.fragment.ConstantLengthFragSet(long_frag_length, long_frag_filename)
long_frag_mover = rosetta.protocols.simple_moves.ClassicFragmentMover(fragset_long, movemap)

fragset_short = rosetta.core.fragment.ConstantLengthFragSet(short_frag_length, short_frag_filename)
short_frag_mover = rosetta.protocols.simple_moves.ClassicFragmentMover(fragset_short, movemap)

# RepeatMover - number of 3/9 fragment pieces to insert during search
insert_long_frag = RepeatMover(long_frag_mover, long_inserts)
insert_short_frag = RepeatMover(short_frag_mover, short_inserts)

core.fragments.ConstantLengthFragSet: finished reading top 200 9mer fragments from file AMHR2_9_fragments.txt
core.fragments.ConstantLengthFragSet: finished reading top 200 3mer fragments from file AMHR2_3_fragments.txt


#### Setting up the MonteCarlo search

In [ ]:
# Making sure the structure is in centroid-only mode for the search
sequence_pose.assign(pose)
to_centroid.apply(sequence_pose)

# Defining what sequence of actions to do between each scoring step
folding_mover = SequenceMover()
folding_mover.add_mover(insert_long_frag)
folding_mover.add_mover(insert_short_frag)

#Define a monte carlo object - centroid_sxfn as the score function
mc = MonteCarlo(sequence_pose, centroid_sfxn, kT)
#A TrialMover applies a Mover and then accepts or rejects the move according to a MonteCarlo object - this mover rejects or accepts a move based on the centroid_sfxn 
trial = TrialMover(folding_mover, mc)

In [ ]:
# Setting up how many cycles of search to do in each trajectory
folding = RepeatMover(trial, cycles)

# Setting up the relax mover for the final stage
fast_relax_mover = rosetta.protocols.relax.FastRelax(full_atom_sfxn)

protocols.relax.RelaxScriptManager: Reading relax scripts list from database.
protocols.relax.RelaxScriptManager: Looking for MonomerRelax2019.txt
protocols.relax.RelaxScriptManager: ================== Reading script file: /usr/local/lib/python3.7/dist-packages/pyrosetta/database/sampling/relax_scripts/MonomerRelax2019.txt ==================
protocols.relax.RelaxScriptManager: repeat %%nrepeats%%
protocols.relax.RelaxScriptManager: coord_cst_weight 1.0
protocols.relax.RelaxScriptManager: scale:fa_rep 0.040
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 0.051
protocols.relax.RelaxScriptManager: min 0.01
protocols.relax.RelaxScriptManager: coord_cst_weight 0.5
protocols.relax.RelaxScriptManager: scale:fa_rep 0.265
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 0.280
protocols.relax.RelaxScriptManager: min 0.01
protocols.relax.RelaxScriptManager: coord_cst_weight 0.0
protocols.relax.RelaxScriptManage

### Running the folding algorithm!

In [ ]:
scores = [0] * (jobs + 1)
scores[0] = centroid_sfxn(sequence_pose)

#Make a directory for storing the outputs

In [ ]:
import shutil

if os.path.isdir(os.path.dirname(job_output)):
    shutil.rmtree(os.path.dirname(job_output), ignore_errors=True)
os.makedirs(os.path.dirname(job_output))
jd = PyJobDistributor(job_output, nstruct=jobs, scorefxn=full_atom_sfxn)

Working on decoy: outputs/AMHR2/decoy_1.pdb


In [ ]:
counter = 0 
while not jd.job_complete:
    #Assign the pose object
    sequence_pose.assign(pose)
    #Set to centroid
    to_centroid.apply(sequence_pose)
    counter += 1
    sequence_pose.pdb_info().name(job_output + '_' + str(counter))

    #start the mc trajectory
    mc.reset(sequence_pose)
    folding.apply(sequence_pose) 
    mc.recover_low(sequence_pose) #get lowest mc pose

    to_full_atom.apply(sequence_pose) #swtich to full atom
    fast_relax_mover.apply(sequence_pose) #apply fast relax
    scores[counter] = full_atom_sfxn(sequence_pose) #score the full atom sequence pose
    #save the output
    jd.output_decoy(sequence_pose)
    sequence_pose.pdb_info().name(job_output + '_' + str(counter) + '_fa')

protocols.relax.FastRelax: CMD: repeat  26127.8  0  0  0.55
protocols.relax.FastRelax: CMD: coord_cst_weight  26127.8  0  0  0.55
protocols.relax.FastRelax: CMD: scale:fa_rep  4524.06  0  0  0.022
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1488 rotamers at 109 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.relax.FastRelax: CMD: repack  728.152  0  0  0.022
protocols.relax.FastRelax: CMD: scale:fa_rep  738.392  0  0  0.02805
protocols.relax.FastRelax: CMD: min  426.824  19.0092  19.0092  0.02805
protocols.relax.FastRelax: CMD: coord_cst_weight  426.824  19.0092  19.0092  0.02805
protocols.relax.FastRelax: CMD: scale:fa_rep  2595.9  19.0092  19.0092  0.14575
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1566 rotamers at 109 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
prot

### Evaluating how close to the original structure the poses after the folding pipeline in Rosetta are. using RMSD

In [ ]:
decoy_poses = [pose_from_pdb(f) for f in glob.glob(job_output + '*.pdb')]

def align_and_get_rmsds(native_pose, decoy_poses):
    rosetta.core.pose.full_model_info.make_sure_full_model_info_is_setup(native_pose)
    rmsds = []
    for p in decoy_poses:
        rosetta.core.pose.full_model_info.make_sure_full_model_info_is_setup(p)
        rmsds += [rosetta.protocols.stepwise.modeler.align.superimpose_with_stepwise_aligner(native_pose, p)]
    return rmsds

rmsds = align_and_get_rmsds(native_pose, decoy_poses)
rmsd_data = []
for i in range(1, len(decoy_poses)):  # print out the job scores
    rmsd_data.append({'structure': decoy_poses[i].pdb_info().name(), 
                      'rmsd': rmsds[i],
                      'energy_score': scores[i]})
    
#Pandas is a library in python that allows you to work with tabular data -> think of as an equivalent of excel sheets 
rmsd_df = pd.DataFrame(rmsd_data)
rmsd_df.sort_values('rmsd')

core.import_pose.import_pose: File 'outputs/AMHR2/decoy_1.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'outputs/AMHR2/decoy_3.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'outputs/AMHR2/decoy_4.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'outputs/AMHR2/decoy_0.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'outputs/AMHR2/decoy_2.pdb' automatically determined to be of type PDB
protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), superimposed on 0 atoms in 1-109 (RMSD 0.0000000)
protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), superimposed on 0 atoms in 1-109 (RMSD 0.0000000)
protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), superimposed on 0 atoms in 1-109 (RMSD 0.0000000)
protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), 

,structure,rmsd,energy_score
0,outputs/AMHR2/decoy_3.pdb,0.0,-77.302528
1,outputs/AMHR2/decoy_4.pdb,0.0,-55.296824
2,outputs/AMHR2/decoy_0.pdb,0.0,-34.297544
3,outputs/AMHR2/decoy_2.pdb,0.0,-65.992991
